In [ ]:
#import tkinter as tk, threading
#mkTkinter
import mkTkinter as tk, threading
import imageio
from PIL import Image, ImageTk

imageio.plugins.ffmpeg.download()
import wave
import os
import datetime
from time import sleep

print("Import done")

In [ ]:
import tkinter as tk
import threading
import imageio
import os
import time
import numpy as np
from PIL import Image, ImageTk
from Interactive import Interactive 
from IPython import embed
import csv
video_name = "TestVideo/HM.mp4" #This is your video file path
video = imageio.get_reader(video_name)
print("start")

def saveDict(dictToSave,fileName):
    w = csv.writer(open(fileName, "w"))
    for key, val in dictToSave.items():
        w.writerow([key, val])

def objectDetectionImagesInFilm():
    
    resultList=[]
    model = Interactive()
    start_time=time.time()
    objectDict= {}
    for index,image in enumerate(video.iter_data()):
        print(index)
        image_frame = Image.fromarray(image)          
        image_frame.save('FRAMES/frame%d.png' % index)
        PATH_TO_TEST_IMAGES_DIR='FRAMES'
        TEST_IMAGE_PATH = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'frame%d.png' % index)]
        results = model.objectDetection(TEST_IMAGE_PATH)
        #print(results)
        results = np.squeeze(results)
        objectDict['frame%d.png' % index] = results
        resultList.append(results)
        if index>10 : break
    saveDict(objectDict,'output.csv')
    #print(objectDict)
    print("The data is saved to a dit now!")
    return (resultList)
    
print(objectDetectionImagesInFilm())


In [ ]:
import csv
dict = {}
for key, val in csv.reader(open("output.csv")):
    dict[key] = val
print(dict['frame1.png'])

In [ ]:

def stream(label):

    frame = 0
    for image in video.iter_data():
        frame += 1                                    #counter to save new frame number
        image_frame = Image.fromarray(image)          
        image_frame.save('FRAMES/frame_%d.png' % frame)      #if you need the frame you can save each frame to hd
        frame_image = ImageTk.PhotoImage(image_frame)
        label.config(image=frame_image)
        label.image = frame_image
        #if frame == 40: break                         #after 40 frames stop, or remove this line for the entire video

if __name__ == "__main__":

    root = tk.Tk()
    my_label = tk.Label(root)
    my_label.pack()
    thread = threading.Thread(target=stream, args=(my_label,))
    thread.daemon = 1
    thread.start()
    root.mainloop()print("hello")

#Plan and whats done now


1. Classify the images using AWS. Will load film and then show image for image. The labels that builds up the image have been run through object detection before and all the frames in the image have been saved. This should be don using AWS to speed up and use GPU. When the images are layed the buttons e.g the objects that you can click on is continuesly updated. This could proabably be done in other ways aswell, where the object detection is done realtime from the video. However the detection only needs to be done once and the program will run more smoothly if it is done seperatly. A model with higher perofmrance could also be used.  To do :

It can now show images and run the film. The film is clickable and we can now start to test it to run and see how it performes when we have classified all the images for objects! 

2. Build something that shows the images and make them clickabe based on the previous classification
3. Includethe image similarity part



In [51]:
import tkinter as tk
import threading
import imageio
import os
import time
import numpy as np
from PIL import Image, ImageTk
from Interactive import Interactive 
from IPython import embed
from collections import namedtuple
import csv
video_name = "TestVideo/HM.mp4" #This is your video file path
video = imageio.get_reader(video_name)
print("start")

class ImageMapper(object):
    #def __init__(self):
    #    pass
    
    def setImage(self, image, img_rects):
        self.width, self.height = image.width(), image.height()
        #print(self.width, self.height)
        self.img_rects = img_rects
        print(img_rects)
        
    def find_rect(self, x, y):
        #print(x,y)
        print(self.img_rects)
        for i, r in enumerate([self.img_rects]):
            if (r.x0 <= x <= r.x1) and (r.y0 <= y <= r.y1):
                return i
        return None

class interactiveVideo():
    image_reacts=[]
    Rect = namedtuple('Rect', 'x0, y0, x1, y1')
    reacts=(Rect(1,2,3,4))

    def __init__(self):
        self.setReact(self.reacts)
        self.root = tk.Tk()
        my_label = tk.Label(self.root)
        my_button = tk.Button(self.root,text='Quit', command=self.printOut())
        my_label.pack()
        thread = threading.Thread(target=self.stream, args=(my_label,my_button,))
        thread.daemon = 1
        thread.start()
        self.root.mainloop()
        print("hello")

    def stream(self,label,my_button):
        self.frame = 0
        self.imagemapper=ImageMapper()
        for image in video.iter_data():
            self.frame += 1                                    #counter to save new frame number
            image_frame = Image.fromarray(image)          
            image_frame.save('FRAMES/frame_%d.png' % self.frame)      #if you need the frame you can save each frame to hd
            frame_image = ImageTk.PhotoImage(image_frame)
            label.config(image=frame_image)
            label.image = frame_image
            label.bind('<Button-1>', self.image_click)
            label.grid(row=1, column=0)
            my_button.grid(row=2, column=0)
            
            #if frame == 40: break                        
            #after 40 frames stop, or remove this line for the entire video
    
    def setReact(self,image_rects):
        for image_react in image_rects:
            self.image_reacts.append(image_react)
            
    def image_click(self, event):
        print("inne")
        print(self.frame)
        
        # Open image need for the check reacts
        # Could skip this later
        PATH_TO_TEST_IMAGES_DIR = 'FRAMES'
        TEST_IMAGE_PATH = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'frame_%d.png' % self.frame)]
        
        # For some reason it only works with self.frame-1 and not the frame it is on.....
        # Why is this.... is the connection still open for the frame it is currently on??? 
        
        image = Image.open('FRAMES/frame_%d.png' % int(self.frame-1))
        print(type(image))
        image = ImageTk.PhotoImage(image)
        print(type(image))
        print("error here? ")
        # Set the reacts for the image 
        my_Rect=self.Rect(0,0,image.width()/2, image.height())
        print(my_Rect)
        self.imagemapper.setImage(image,my_Rect)
        # When we hit what happens
        hit = self.imagemapper.find_rect(event.x, event.y)
        #self.msg_text.set('{} clicked'.format('nothing' if hit is None else
        #                                      'rect[{}]'.format(hit)))
        print(('{} clicked'.format('nothing' if hit is None else
                                              'rect[{}]'.format(hit))))
        #print("the label of the hit is {}".format(self.labels[hit]))
        
        # This part was the cropp part it also worked with the similaritys i think 
        # This part needs further development later on . 
        
        #num=self.labels[hit]
        #image_object=self.image_reacts[hit]
        #crop_area= (image_object.x0,image_object.y0,image_object.x1,image_object.y1)
        #self.cropImage(crop_area,self.TEST_IMAGE_PATH[0])
        
        #self.openLink(num)
            
    def printOut(self):
        print("hello")
            
if __name__ == "__main__":
    print("Hello world")
    a=interactiveVideo()
    #root = tk.Tk()
    #my_label = tk.Label(root)
    #my_label.pack()
    #thread = threading.Thread(target=stream, args=(my_label,))
    #thread.daemon = 1
    #thread.start()
    #root.mainloop()print("hello")

Exception in thread Thread-71:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-50-8cb5572459df>", line 58, in stream
    frame_image = ImageTk.PhotoImage(image_frame)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/PIL/ImageTk.py", line 112, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/tkinter/__init__.py", line 3393, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/tkinter/__init__.py", line 3334, in __init__
    raise RuntimeError('Too early to create image')
RuntimeError: Too early to 

start
Hello world
hello
inne
36
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.ImageTk.PhotoImage'>
error here? 
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
rect[0] clicked
inne
51
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.ImageTk.PhotoImage'>
error here? 
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
rect[0] clicked
inne
67
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.ImageTk.PhotoImage'>
error here? 
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
rect[0] clicked
inne
88
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.ImageTk.PhotoImage'>
error here? 
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
Rect(x0=0, y0=0, x1=320.0, y1=360)
rect[0] clicked
inne
104
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.ImageTk.PhotoImage'>
error here? 
Rect(x0=0, 

Exception in thread Thread-73:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-51-5e9306260770>", line 58, in stream
    frame_image = ImageTk.PhotoImage(image_frame)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/PIL/ImageTk.py", line 112, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/tkinter/__init__.py", line 3393, in __init__
    Image.__init__(self, 'photo', name, cnf, master, **kw)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/tkinter/__init__.py", line 3334, in __init__
    raise RuntimeError('Too early to create image')
RuntimeError: Too early to 

In [3]:
print("Hello ")

Hello 


In [21]:
frame=5
myimage=('frame_%d.png'% frame)
print(myimage)

frame_5.png


In [24]:
frame=5
PATH_TO_TEST_IMAGES_DIR = 'FRAMES'
TEST_IMAGE_PATH = [ os.path.join(PATH_TO_TEST_IMAGES_DIR,'frame_%d.png'% frame ) ]
                                         